<a href="https://colab.research.google.com/github/Wason1/Multum/blob/main/Multum_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Multum 

## Multum 2.7 Missing DNUMS
time: 0.5 seconds
### Set initial variables

In [141]:
#excel_file_path = r'\\whoffice\shared\EMR\BAU\Audit Spreadsheets\Multum audits\Multum Processing 6 - catalogue-Multum_Alignment.xlsx'
excel_file_path = r'C:\storage\data\Multum Processing 6 - catalogue-Multum_Alignment.xlsx'
output_file_path = r'C:\storage\data\dataout.xlsx'
PROD = 'P2031'
NONPROD = 'C2031'
# Below are a list of words that have zero weighting towards the matching algorithm between multum and our catelog
no_match_lst = ['topical', 'nasal']

## Import Libraries and read Multum 6
### Time to execute block below
* over network: 5 mins 15 seconds (approx 320 seconds)
* from local SSD: 17 seconds

In [157]:
# Import Libraries
import pandas as pd
import numpy as np
import openpyxl as pyxl
#import os

# Read Multum 6
m = pd.read_excel(excel_file_path, sheet_name=None)
print('ignore warnings above this line______')

# Show sheet names in Multum 6
print('---EXCEL SHEET NAMES---')
for x in m.keys():
    print(x)
print('-----------------------')

if PROD not in m.keys():
    print('Your itnitial conditions are not correctly set')
else:
    print('You may continue')

C:\Users\jason\AppData\Local\Programs\Python\Python39\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)


ignore warnings above this line______
---EXCEL SHEET NAMES---
C2031
P2031
syn-Mul compare extract script
current Multum
current Multum extract script
Multum build exceptions
obsolete Multum
obsolete Multum extract script
Multum SRs
-----------------------
You may continue


## 2.7.4.
time: 0.1 seconds
The code block below does the equivalent of:

"On the selected environment’s worksheet, filter column F [PRIMARY_MNEMONIC] to show only light yellow coloured cells. These are active primaries that do not have a DNUM."

In [165]:
# Filter for active primaries that do not have a DNUM. and put in table p_1
# Equivalent to filtering for 'yellow' on the 'PRIMARY_MNEMONIC' column (in the excel tool)
prod_data = m[PROD].fillna('', inplace = False) # blank cells are literally blank strings '' now.
prod_data = prod_data.astype(str)
prod_data.columns = prod_data.columns.str.replace(' ','_') # Changing column names to get rid of spaces
prod_data.columns = prod_data.columns.str.replace("'",'')
# prod_active_primaries = prod.query(" SYNONYM_TYPE == 'Primary' & SYNONYM_ACTIVE == 1 & PRIMARY_DNUM == '' ")
p_1 = prod_data
p_1 = p_1.loc[p_1['SYNONYM_TYPE'] == 'Primary']
p_1 = p_1.loc[p_1['SYNONYM_ACTIVE'] == '1']
p_1 = p_1.loc[p_1['PRIMARY_DNUM'] == '']

## Missing DNUMS
time: 0.6 seconds

## The block below will get all the 'current Multum' DNUMS that are not associated with items in our prod catalogue

In [173]:

# Rename the table, make all data in the cells strings and rename columns
m_1 = m['current Multum'].astype(str)
m_1.columns = m_1.columns.str.replace(' ','_') # Changing column names to get rid of spaces
m_1.columns = m_1.columns.str.replace("'",'') # Changing column names to get rid of thes
# Filtering Dataframe
# the stuff below does the same as this formula in the current Multum sheet. Every line filters the data some more
# Formula: =AND($B1>0,$I1=0,$J1=0,$M1<>1,$P1<>1,$S1<>1,$V1<>1,$Y1<>1,$AB1<>1,$AE1<>1)
m_1 = m_1.loc[m_1['MOCL_SYNONYM_TYPE'] == 'Primary']
m_1 = m_1.loc[m_1['MOCL_DNUM'].str.startswith('MUL')]
m_1 = m_1.loc[m_1['SYNONYM_BUILT']  == '0' ]
m_1 = m_1.loc[m_1['generic_manufacturer_do_not_build_rule']  != '1' ]
m_1 = m_1.loc[m_1['multi-ingredient_primary_do_not_build_rule']  != '1' ]
m_1 = m_1.loc[m_1['volume_dosed_primary_do_not_build_rule']  != '1' ]
m_1 = m_1.loc[m_1['flu_vax_primary_do_not_build_rule']  != '1' ]
m_1 = m_1.loc[m_1['non-substitutable_primary_do_not_build_rule']  != '1' ]
m_1 = m_1.loc[m_1['PBS_do_not_build_rule']  != '1' ]
m_1 = m_1.loc[m_1['miscellaneous_do_not_build']  != '1' ]
m_1.reset_index(drop=True, inplace=True) # Reindex the data (optional)

# Rename the table, make all data in the cells strings and rename columns
m_2 = m['current Multum'].astype(str)
m_2.columns = m_2.columns.str.replace(' ','_') # Changing column names to get rid of spaces
m_2.columns = m_2.columns.str.replace("'",'')
# Filtering Dataframe with the second conditional formula
# the stuff below does the same as this formula in the current Multum sheet. Every line filters the data some more
# Formula: =AND($I1=0,$AF1=1)
m_2 = m_2.loc[m_2['SYNONYM_BUILT']  == '0' ]
m_2 = m_2.loc[m_2['miscellaneous_do_build']  == '1' ]
m_2.reset_index(drop=True, inplace=True) # Reindex the data (optional)

# Merge m_1 and m_2
free_mul_primaries = pd.concat([m_1, m_2], ignore_index=True, sort=True)
# Get rid of duplicates
free_mul_primaries.drop_duplicates(inplace = True) 
free_mul_primaries.reset_index(drop=True, inplace=True) # Reindex the data (optional)

## Generate Excel File so user can choose associations.
time: 0.5 seconds

In [189]:
# Unmatched Primaries in Prod
p_2 = p_1.filter(['PRIMARY_MNEMONIC'])
p_2.reset_index(drop=True, inplace=True)

# Create a df for Multum - Catelog matching spreadsheet
column1 = 'PRIMARY_MNEMONIC' #cerner
column2 = 'MOCL_PRIMARY_MNEMONIC' #multum
column3 = 'MOCL_DNUM' # Multum
column4 = 'MOCL_CNUM' # Multum
#df = pd.DataFrame(columns=[column1, column2])
df = pd.DataFrame(columns = [column1,column2, column3, column4])

# Array of all unmatched Multum Primaries
mul_p = free_mul_primaries.filter(items = [column2, column3, column4])

# Matching Algorithm
for index, row in p_2.iterrows():
    cernername = row[column1]
    current_key_words = cernername.split(' ')
    for i, r in mul_p.iterrows():
        multumname = r[column2]
        words_multumname = multumname.split(' ')
        # Remove the words that should have no impact on matching such as 'topical'
        words_y = [i for i in words_multumname if i not in no_match_lst]
        # Checking if any unallocated catelog primaries match with any unallocated multum primaries and add to table
        if bool(set(current_key_words) & set(words_y)):
            #make a mini table
            data = {column1: [cernername], column2: [multumname], column3: [r[column3]], column4: [r[column4]]}  
            df_temp = pd.DataFrame(data)
            # append to main table
            df = pd.concat([df, df_temp], ignore_index=True)

# Add match column
column0 = 'MATCH? "x"'
df[column0]=''
# reorder columns
column_order = [column0,column1,column2,column3,column4]
df = df[column_order]

# Replace duplicates in the PRIMARY_MNEMONIC column with blanks, keep first
prev_row = 'x'
for index, row in df.iterrows():
    if row['PRIMARY_MNEMONIC'] == prev_row:
        df.loc[index, 'PRIMARY_MNEMONIC'] = ''
    prev_row = row['PRIMARY_MNEMONIC']

df.to_excel(output_file_path)

## Read the excel sheet

In [ ]:
df_back_in = pd.read_excel(output_file_path, sheet_name=None)